In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
batch_size_pipeline = 4
learning_rate_pipeline = 0.08/100
warmup_learning_rate_pipeline = 0.026666/100
num_steps_training = 1000
PRETRAINED_MODEL_GG_DRIVE_PATH = '/content/drive/MyDrive/workspace_20210831_185535/model_20210831_185535'

# 0. Setup Paths

In [4]:
import datetime
import pytz

def shorten_time(date):
    dt_year = date[0:4]
    dt_month = date[5:7]
    dt_day = date[8:10]
    dt_hr = date[11:13]
    dt_min = date[14:16]
    dt_sec = date[17:19]
    dt_full = "{}{}{}_{}{}{}".format(dt_year,dt_month,dt_day,dt_hr,dt_min,dt_sec)
    return dt_full

start_date = str(datetime.datetime.now(pytz.timezone('Asia/Bangkok')))[:19]
start_date_short = shorten_time(start_date)
print(start_date)
print(start_date_short)

2021-09-01 00:09:53
20210901_000953


In [5]:
gg_drive_path = "/content/drive/MyDrive"

In [6]:
models_drive_path = "{}/workspace_{}".format(gg_drive_path,start_date_short)
print(models_drive_path)

/content/drive/MyDrive/workspace_20210901_000953


In [7]:
import os

In [8]:
CUSTOM_MODEL_NAME = 'my_ssd_custom'
PRETRAINED_MODEL_NAME = PRETRAINED_MODEL_GG_DRIVE_PATH[len(PRETRAINED_MODEL_GG_DRIVE_PATH)-21 :]
TF_RECORD_SCRIPT_NAME = 'generate_tfrecord.py'
LABEL_MAP_NAME = 'label_map.pbtxt'

In [9]:
paths = {
    'WORKSPACE_PATH': os.path.join('Tensorflow', 'workspace'),
    'SCRIPTS_PATH': os.path.join('Tensorflow','scripts'),
    'APIMODEL_PATH': os.path.join('Tensorflow','models'),
    'ANNOTATION_PATH': os.path.join('Tensorflow', 'workspace','annotations'),
    'IMAGE_PATH': os.path.join('Tensorflow', 'workspace','images'),
    'PRETRAINED_MODEL_PATH': os.path.join('Tensorflow', 'workspace','pre-trained-models'),
    'PROTOC_PATH':os.path.join('Tensorflow','protoc'),
    'MODEL_PATH': os.path.join(models_drive_path),
    'CHECKPOINT_PATH': os.path.join(models_drive_path,CUSTOM_MODEL_NAME), 
    'OUTPUT_PATH': os.path.join(models_drive_path,CUSTOM_MODEL_NAME, 'export'), 
    'TFJS_PATH':os.path.join(models_drive_path,CUSTOM_MODEL_NAME, 'tfjsexport'), 
    'TFLITE_PATH':os.path.join(models_drive_path,CUSTOM_MODEL_NAME, 'tfliteexport'), 
 }

In [10]:
files = {
    'PIPELINE_CONFIG':os.path.join(models_drive_path, CUSTOM_MODEL_NAME, 'pipeline.config'),
    'TF_RECORD_SCRIPT': os.path.join(paths['SCRIPTS_PATH'], TF_RECORD_SCRIPT_NAME), 
    'LABELMAP': os.path.join(paths['ANNOTATION_PATH'], LABEL_MAP_NAME)
}

In [11]:
for path in paths.values():
    if not os.path.exists(path):
        if os.name == 'posix':
            !mkdir -p {path}
        if os.name == 'nt':
            !mkdir {path}

# 1. Install TF Object Dectection API

In [12]:
# https://www.tensorflow.org/install/source_windows

In [13]:
if os.name=='nt':
    !pip install wget
    import wget

In [14]:
if not os.path.exists(os.path.join(paths['APIMODEL_PATH'], 'research', 'object_detection')):
    !git clone https://github.com/tensorflow/models {paths['APIMODEL_PATH']}

Cloning into 'Tensorflow/models'...
remote: Enumerating objects: 61008, done.
remote: Counting objects: 100% (57/57), done.
remote: Compressing objects: 100% (36/36), done.
remote: Total 61008 (delta 22), reused 48 (delta 20), pack-reused 60951
Receiving objects: 100% (61008/61008), 573.98 MiB | 30.38 MiB/s, done.
Resolving deltas: 100% (42486/42486), done.


In [15]:
# Install Tensorflow Object Detection 
if os.name=='posix':  
    !apt-get install protobuf-compiler
    !cd Tensorflow/models/research && protoc object_detection/protos/*.proto --python_out=. && cp object_detection/packages/tf2/setup.py . && python -m pip install . 
    
if os.name=='nt':
    url="https://github.com/protocolbuffers/protobuf/releases/download/v3.15.6/protoc-3.15.6-win64.zip"
    wget.download(url)
    !move protoc-3.15.6-win64.zip {paths['PROTOC_PATH']}
    !cd {paths['PROTOC_PATH']} && tar -xf protoc-3.15.6-win64.zip
    os.environ['PATH'] += os.pathsep + os.path.abspath(os.path.join(paths['PROTOC_PATH'], 'bin'))   
    !cd Tensorflow/models/research && protoc object_detection/protos/*.proto --python_out=. && copy object_detection\\packages\\tf2\\setup.py setup.py && python setup.py build && python setup.py install
    !cd Tensorflow/models/research/slim && pip install -e . 

Reading package lists... Done
Building dependency tree       
Reading state information... Done
protobuf-compiler is already the newest version (3.0.0-9.1ubuntu1).
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'apt autoremove' to remove it.
0 upgraded, 0 newly installed, 0 to remove and 40 not upgraded.
Processing /content/Tensorflow/models/research
  DEPRECATION: A future pip version will change local packages to be built in-place without first copying to a temporary directory. We recommend you use --use-feature=in-tree-build to test your packages with this new behavior before it becomes the default.
   pip 21.3 will remove support for this functionality. You can find discussion regarding this at https://github.com/pypa/pip/issues/7555.
     |████████████████████████████████| 9.8 MB 8.5 MB/s 
     |████████████████████████████████| 352 kB 52.0 MB/s 
     |████████████████████████████████| 1.8 MB 44.8 MB/s 
     |███████████████

In [16]:
print(os.path.join(paths['APIMODEL_PATH'], 'research', 'object_detection', 'builders', 'model_builder_tf2_test.py'))

Tensorflow/models/research/object_detection/builders/model_builder_tf2_test.py


In [17]:
VERIFICATION_SCRIPT = os.path.join(paths['APIMODEL_PATH'], 'research', 'object_detection', 'builders', 'model_builder_tf2_test.py')
# Verify Installation
!python {VERIFICATION_SCRIPT}

2021-08-31 17:11:05.112974: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-08-31 17:11:05.856268: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-08-31 17:11:05.857138: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
Running tests under Python 3.7.11: /usr/bin/python3
[ RUN      ] ModelBuilderTF2Test.test_create_center_net_deepmac
2021-08-31 17:11:05.874902: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021

In [18]:
import object_detection

#2. Copy pretrained model from GG drive to workspace on GG colab

In [19]:
!cp -r {PRETRAINED_MODEL_GG_DRIVE_PATH} {paths['PRETRAINED_MODEL_PATH']}

# 3. Create Label Map

In [20]:
labels = [{'name':'car', 'id':1}]

with open(files['LABELMAP'], 'w') as f:
    for label in labels:
        f.write('item { \n')
        f.write('\tname:\'{}\'\n'.format(label['name']))
        f.write('\tid:{}\n'.format(label['id']))
        f.write('}\n')

# 4. Create TF records

In [21]:
# Unrar image files from drive and locate in /content/Tensorflow/workspace/images
!unrar x /content/drive/MyDrive/car_coco_2017.rar -d /content/Tensorflow/workspace/images

Streaming output truncated to the last 5000 lines.
Extracting  /content/Tensorflow/workspace/images/train/000000425691.xml      79%  OK 
Extracting  /content/Tensorflow/workspace/images/train/000000425743.jpg      79%  OK 
Extracting  /content/Tensorflow/workspace/images/train/000000425743.xml      79%  OK 
Extracting  /content/Tensorflow/workspace/images/train/000000425746.jpg      79%  OK 
Extracting  /content/Tensorflow/workspace/images/train/000000425746.xml      79%  OK 
Extracting  /content/Tensorflow/workspace/images/train/000000425807.jpg      79%  OK 
Extracting  /content/Tensorflow/workspace/images/train/000000425807.xml      79%  OK 
Extracting  /content/Tensorflow/workspace/images/train/000000425847.jpg      79%  OK 
Extracting  /content/Tensorflow/workspace/images/train/000000425847.xml      79%  OK 
Extracting  /content/Tensorflow/workspace/images/train/000000425864.jpg      79%  OK 

In [22]:
if not os.path.exists(files['TF_RECORD_SCRIPT']):
    !git clone https://github.com/nicknochnack/GenerateTFRecord {paths['SCRIPTS_PATH']}

Cloning into 'Tensorflow/scripts'...
remote: Enumerating objects: 3, done.
remote: Counting objects: 100% (3/3), done.
remote: Compressing objects: 100% (2/2), done.
remote: Total 3 (delta 0), reused 1 (delta 0), pack-reused 0
Unpacking objects: 100% (3/3), done.


In [23]:
!python {files['TF_RECORD_SCRIPT']} -x {os.path.join(paths['IMAGE_PATH'], 'train')} -l {files['LABELMAP']} -o {os.path.join(paths['ANNOTATION_PATH'], 'train.record')} 
!python {files['TF_RECORD_SCRIPT']} -x {os.path.join(paths['IMAGE_PATH'], 'test')} -l {files['LABELMAP']} -o {os.path.join(paths['ANNOTATION_PATH'], 'test.record')} 

Successfully created the TFRecord file: Tensorflow/workspace/annotations/train.record
Successfully created the TFRecord file: Tensorflow/workspace/annotations/test.record


# 5. Copy Model Config to Training Folder

In [24]:
if os.name =='posix':
    !cp {os.path.join(paths['PRETRAINED_MODEL_PATH'], PRETRAINED_MODEL_NAME, 'pipeline.config')} {os.path.join(paths['CHECKPOINT_PATH'])}
if os.name == 'nt':
    !copy {os.path.join(paths['PRETRAINED_MODEL_PATH'], PRETRAINED_MODEL_NAME, 'pipeline.config')} {os.path.join(paths['CHECKPOINT_PATH'])}

# 6. Update Config For Transfer Learning

In [25]:
import tensorflow as tf
from object_detection.utils import config_util
from object_detection.protos import pipeline_pb2
from google.protobuf import text_format

In [26]:
config = config_util.get_configs_from_pipeline_file(files['PIPELINE_CONFIG'])

In [27]:
config

{'eval_config': metrics_set: "coco_detection_metrics"
 use_moving_averages: false,
 'eval_input_config': label_map_path: "Tensorflow/workspace/annotations/label_map.pbtxt"
 shuffle: false
 num_epochs: 1
 tf_record_input_reader {
   input_path: "Tensorflow/workspace/annotations/test.record"
 },
 'eval_input_configs': [label_map_path: "Tensorflow/workspace/annotations/label_map.pbtxt"
 shuffle: false
 num_epochs: 1
 tf_record_input_reader {
   input_path: "Tensorflow/workspace/annotations/test.record"
 }
 ],
 'model': ssd {
   num_classes: 1
   image_resizer {
     fixed_shape_resizer {
       height: 320
       width: 320
     }
   }
   feature_extractor {
     type: "ssd_mobilenet_v2_fpn_keras"
     depth_multiplier: 1.0
     min_depth: 16
     conv_hyperparams {
       regularizer {
         l2_regularizer {
           weight: 3.9999998989515007e-05
         }
       }
       initializer {
         random_normal_initializer {
           mean: 0.0
           stddev: 0.00999999977648258

In [28]:
pipeline_config = pipeline_pb2.TrainEvalPipelineConfig()
with tf.io.gfile.GFile(files['PIPELINE_CONFIG'], "r") as f:                                                                                                                                                                                                                     
    proto_str = f.read()                                                                                                                                                                                                                                          
    text_format.Merge(proto_str, pipeline_config)  

In [29]:
pipeline_config.model.ssd.num_classes = len(labels)
pipeline_config.train_config.batch_size = batch_size_pipeline
pipeline_config.train_config.optimizer.momentum_optimizer.learning_rate.cosine_decay_learning_rate.learning_rate_base = learning_rate_pipeline
pipeline_config.train_config.optimizer.momentum_optimizer.learning_rate.cosine_decay_learning_rate.warmup_learning_rate = warmup_learning_rate_pipeline
pipeline_config.train_config.fine_tune_checkpoint = os.path.join(paths['PRETRAINED_MODEL_PATH'], PRETRAINED_MODEL_NAME, 'checkpoint', 'ckpt-0')
pipeline_config.train_config.fine_tune_checkpoint_type = "detection"
pipeline_config.train_input_reader.label_map_path= files['LABELMAP']
pipeline_config.train_input_reader.tf_record_input_reader.input_path[:] = [os.path.join(paths['ANNOTATION_PATH'], 'train.record')]
pipeline_config.eval_input_reader[0].label_map_path = files['LABELMAP']
pipeline_config.eval_input_reader[0].tf_record_input_reader.input_path[:] = [os.path.join(paths['ANNOTATION_PATH'], 'test.record')]

In [30]:
config_text = text_format.MessageToString(pipeline_config)                                                                                                                                                                                                        
with tf.io.gfile.GFile(files['PIPELINE_CONFIG'], "wb") as f:                                                                                                                                                                                                                     
    f.write(config_text)   

# 7. Train the model

In [31]:
TRAINING_SCRIPT = os.path.join(paths['APIMODEL_PATH'], 'research', 'object_detection', 'model_main_tf2.py')

In [32]:
command = "python {} --model_dir={} --pipeline_config_path={} --num_train_steps={}".format(TRAINING_SCRIPT, paths['CHECKPOINT_PATH'],files['PIPELINE_CONFIG'],num_steps_training)

In [33]:
print(command)

python Tensorflow/models/research/object_detection/model_main_tf2.py --model_dir=/content/drive/MyDrive/workspace_20210901_000953/my_ssd_custom --pipeline_config_path=/content/drive/MyDrive/workspace_20210901_000953/my_ssd_custom/pipeline.config --num_train_steps=1000


In [34]:
!{command}

2021-08-31 17:14:21.594871: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-08-31 17:14:21.617459: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-08-31 17:14:21.618317: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-08-31 17:14:21.626198: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-08-31 17:14:21.627069: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from S

# 8. Freezing the Graph

In [35]:
FREEZE_SCRIPT = os.path.join(paths['APIMODEL_PATH'], 'research', 'object_detection', 'exporter_main_v2.py ')

In [36]:
command = "python {} --input_type=image_tensor --pipeline_config_path={} --trained_checkpoint_dir={} --output_directory={}".format(FREEZE_SCRIPT ,files['PIPELINE_CONFIG'], paths['CHECKPOINT_PATH'], paths['OUTPUT_PATH'])

In [37]:
print(command)

python Tensorflow/models/research/object_detection/exporter_main_v2.py  --input_type=image_tensor --pipeline_config_path=/content/drive/MyDrive/workspace_20210901_000953/my_ssd_custom/pipeline.config --trained_checkpoint_dir=/content/drive/MyDrive/workspace_20210901_000953/my_ssd_custom --output_directory=/content/drive/MyDrive/workspace_20210901_000953/my_ssd_custom/export


In [38]:
!{command}

2021-08-31 17:19:35.169845: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-08-31 17:19:35.186141: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-08-31 17:19:35.187008: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-08-31 17:19:35.199326: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-08-31 17:19:35.200189: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from S

# 9. Zip model and copy to GG Drive

In [39]:
!cp -r {paths['OUTPUT_PATH']} {paths['MODEL_PATH']}

In [40]:
os.rename("{}/{}".format(paths['MODEL_PATH'],"export"),"{}/model_{}".format(paths['MODEL_PATH'],start_date_short))

In [41]:
export_model_path = "{}/model_{}".format(paths['MODEL_PATH'],start_date_short)
export_model_zip = "{}/model_{}.zip".format(paths['MODEL_PATH'],start_date_short)

In [42]:
!zip -r {export_model_zip} {export_model_path}

  adding: content/drive/MyDrive/workspace_20210901_000953/model_20210901_000953/ (stored 0%)
  adding: content/drive/MyDrive/workspace_20210901_000953/model_20210901_000953/checkpoint/ (stored 0%)
  adding: content/drive/MyDrive/workspace_20210901_000953/model_20210901_000953/checkpoint/ckpt-0.data-00000-of-00001 (deflated 8%)
  adding: content/drive/MyDrive/workspace_20210901_000953/model_20210901_000953/checkpoint/ckpt-0.index (deflated 80%)
  adding: content/drive/MyDrive/workspace_20210901_000953/model_20210901_000953/checkpoint/checkpoint (deflated 41%)
  adding: content/drive/MyDrive/workspace_20210901_000953/model_20210901_000953/saved_model/ (stored 0%)
  adding: content/drive/MyDrive/workspace_20210901_000953/model_20210901_000953/saved_model/variables/ (stored 0%)
  adding: content/drive/MyDrive/workspace_20210901_000953/model_20210901_000953/saved_model/variables/variables.data-00000-of-00001 (deflated 8%)
  adding: content/drive/MyDrive/workspace_20210901_000953/model_20210

# 10. Notification via Email

In [43]:
import datetime
import pytz

finish_date = str(datetime.datetime.now(pytz.timezone('Asia/Bangkok')))[:19]
finish_date_short = shorten_time(finish_date)

print(finish_date)
print(finish_date_short)

2021-09-01 00:20:42
20210901_002042


In [44]:
sd_year = int(start_date_short[0:4])
sd_month = int(start_date_short[4:6])
sd_day = int(start_date_short[6:8])
sd_hour = int(start_date_short[9:11])
sd_min = int(start_date_short[11:13])
sd_sec = int(start_date_short[13:15])

fd_year = int(finish_date_short[0:4])
fd_month = int(finish_date_short[4:6])
fd_day = int(finish_date_short[6:8])
fd_hour = int(finish_date_short[9:11])
fd_min = int(finish_date_short[11:13])
fd_sec = int(finish_date_short[13:15])

In [45]:
start = datetime.datetime(year=sd_year, month=sd_month, day=sd_day, hour=sd_hour, minute=sd_min,second=sd_sec)
end = datetime.datetime(year=fd_year, month=fd_month, day=fd_day, hour=fd_hour, minute=fd_min,second=fd_sec)
diff = end - start

In [46]:
import smtplib
from email.message import EmailMessage

msg_subject = "[GG Colab] Model {}".format(start_date_short)
msg_content = "It take {} for training.".format(diff)

msg = EmailMessage()
msg['Subject'] = msg_subject
msg['From'] = '*'
msg['To']='ptmhoang97@gmail.com'
msg.set_content(msg_content)

server = smtplib.SMTP_SSL('smtp.gmail.com',465)
server.login("ptmhoang97@gmail.com","euclxykrqkkrquqz")
server.send_message(msg)
server.quit()

(221, b'2.0.0 closing connection l15sm9969034iow.4 - gsmtp')